### Processing pipeline for RAVDESS Database

#### Enviroment preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

%cd /content

!rm -rf automatic-pain-recognition

print(f"Obecne położenie: {os.getcwd()}")

In [ ]:
USERNAME = 'alicka33'
REPO_NAME = 'automatic-pain-recognition'

REPO_URL = f"https://github.com/{USERNAME}/{REPO_NAME}.git"

print("Rozpoczynam klonowanie...")

!git clone {REPO_URL}

%cd $REPO_NAME

In [ ]:
!pip install -r requirements.txt

In [ ]:
import sys
import os

sys.path.append(os.getcwd())
print(f"Dodano do ścieżki Pythona: {os.getcwd()}")

#### Dataset processing

In [ ]:
from pathlib import Path
import pandas as pd

from data_preparation.processing_pipeline_mediapipe import (
    load_reference_keypoints,
    video_to_feature_sequences,
    visualize_frontalized_points,
)
from data_preparation.process_dataset import ProcessingConfig, process_dataframe_to_npy

print("Modules imported")

In [ ]:

cfg = ProcessingConfig(
    dataset_subdir='data/RAVDESS/',
    processed_subdir='data/RAVDESS_processed_478_xyz_frontalized/',
)

print("Data dir:", cfg.data_dir)
print("Processed dir:", cfg.processed_data_dir)
print("Reference keypoints path:", cfg.reference_path)

In [ ]:
train_df = pd.read_csv(os.path.join(cfg.data_dir, 'train.csv'))
val_df = pd.read_csv(os.path.join(cfg.data_dir, 'val.csv'))
test_df = pd.read_csv(os.path.join(cfg.data_dir, 'test.csv'))

print("Rows (train, val, test):", len(train_df), len(val_df), len(test_df))
train_df.head()

In [ ]:
ref_kp, ref_ok = load_reference_keypoints(cfg.reference_path)
print("Reference keypoints loaded:", ref_ok)
if not ref_ok:
    print("Frontalization will be disabled (or set cfg.reference_keypoints_path to a valid file).")

In [ ]:
sample = train_df.iloc[0]
sample_video_path = os.path.join(cfg.data_dir, sample['video_path'])
print("Visualizing:", sample_video_path)

video_to_feature_sequences(
    str(sample_video_path),
    frame_skip=3,
    reference_keypoints_3d=ref_kp,
    use_frontalization=(cfg.use_frontalization and ref_ok),
    visualize=True
)

In [ ]:
process_dataframe_to_npy(train_df, 'train', video_to_feature_sequences, config=cfg, frame_skip=3, visualize=False)
process_dataframe_to_npy(val_df, 'val', video_to_feature_sequences, config=cfg, frame_skip=3, visualize=False)
process_dataframe_to_npy(test_df, 'test', video_to_feature_sequences, config=cfg, frame_skip=3, visualize=False)